In [10]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, LSTM
#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
file_paths_train = [
    "/content/body_acc_x_train.txt", "/content/body_acc_y_train.txt", "/content/body_acc_z_train.txt",
    "/content/body_gyro_x_train.txt", "/content/body_gyro_y_train.txt", "/content/body_gyro_z_train.txt",
    "/content/total_acc_x_train.txt", "/content/total_acc_y_train.txt", "/content/total_acc_z_train.txt"
]

# combine axes (x, y, z) for a signal
def combine_axes(file_paths, signal_type):
    signal_files = [fp for fp in file_paths if signal_type in fp]
    signal_data = []

    for file_path in sorted(signal_files):
        data = np.loadtxt(file_path)
        signal_data.append(data)

    # Stack x, y, z into  3D array
    combined_signal = np.stack(signal_data, axis=-1)
    return combined_signal

# Combine for each signal type (train data)
body_acc_train = combine_axes(file_paths_train, "body_acc")
body_gyro_train = combine_axes(file_paths_train, "body_gyro")
total_acc_train = combine_axes(file_paths_train, "total_acc")

# Combine all signals (train data)
combined_data_train_x = np.concatenate([body_acc_train, body_gyro_train, total_acc_train], axis=-1)
print("Combined training data shape:", combined_data_train_x.shape)
#////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
file_paths_test = [
    "/content/body_acc_x_test.txt", "/content/body_acc_y_test.txt", "/content/body_acc_z_test.txt",
    "/content/body_gyro_x_test.txt", "/content/body_gyro_y_test.txt", "/content/body_gyro_z_test.txt",
    "/content/total_acc_x_test.txt", "/content/total_acc_y_test.txt", "/content/total_acc_z_test.txt"
]

# Combine  (test data)
body_acc_test = combine_axes(file_paths_test, "body_acc")
body_gyro_test = combine_axes(file_paths_test, "body_gyro")
total_acc_test = combine_axes(file_paths_test, "total_acc")

combined_data_test_x = np.concatenate([body_acc_test, body_gyro_test, total_acc_test], axis=-1)
print("Combined test data shape:", combined_data_test_x.shape)

# Load Labels
y_train = np.loadtxt("/content/y_train.txt").astype(int) - 1
y_test = np.loadtxt("/content/y_test.txt").astype(int) - 1

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Check dataset shapes
print("X_train shape:", combined_data_train_x.shape)
print("X_test shape:", combined_data_test_x.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

#  model training and evaluation function
def train_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

    # Evaluate
    acc = accuracy_score(y_true, y_pred_classes)
    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')
    f1 = f1_score(y_true, y_pred_classes, average='weighted')

    print(f"Accuracy: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# 1D-CNN Model
def create_cnn(input_shape, num_classes):
    model = Sequential([
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape),
        Dropout(0.5),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# MLP Model
def create_mlp(input_shape, num_classes):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(100, activation='relu'),
        Dropout(0.5),
        Dense(50, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

# LSTM Model
def create_lstm(input_shape, num_classes):
    model = Sequential([
        LSTM(100, input_shape=input_shape, return_sequences=True),
        LSTM(50),
        Dense(50, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = combined_data_train_x.shape[1:]
num_classes = y_train.shape[1]

# Train and Evaluate using CNN
print("\nTraining 1D-CNN Model")
cnn_model = create_cnn(input_shape, num_classes)
train_evaluate_model(cnn_model, combined_data_train_x, y_train, combined_data_test_x, y_test)

# Train and Evaluate using MLP
print("\nTraining MLP Model")
mlp_model = create_mlp(input_shape, num_classes)
train_evaluate_model(mlp_model, combined_data_train_x, y_train, combined_data_test_x, y_test)

# Train and Evaluate using LSTM
print("\nTraining LSTM Model")
lstm_model = create_lstm(input_shape, num_classes)
train_evaluate_model(lstm_model, combined_data_train_x, y_train, combined_data_test_x, y_test)

Combined training data shape: (7352, 128, 9)
Combined test data shape: (2947, 128, 9)
X_train shape: (7352, 128, 9)
X_test shape: (2947, 128, 9)
y_train shape: (7352, 6)
y_test shape: (2947, 6)

Training 1D-CNN Model


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.7457 - loss: 0.6258
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9132 - loss: 0.2173
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9496 - loss: 0.1296
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9472 - loss: 0.1215
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9555 - loss: 0.1186
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9600 - loss: 0.0961
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.9539 - loss: 0.1032
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9586 - loss: 0.0926
Epoch 9/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9607 - loss: 0.0917
Epoch 10/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.9606 - loss: 0.0876
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
Accuracy: 0.8945, Precision: 0.9004, Recall: 0.8945, F1 Score: 0.8947

Training MLP Model


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5527 - loss: 1.1412
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8132 - loss: 0.5128
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8533 - loss: 0.3804
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8714 - loss: 0.3404
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8781 - loss: 0.3159
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8975 - loss: 0.2710
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9030 - loss: 0.2556
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9005 - loss: 0.2498
Epoch 9/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9080 - loss: 0.2465
Epoch 10/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9020 - loss: 0.2433
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.8809, Precision: 0.8822, Recall: 0.8809, F1 Score: 0.8807

Training LSTM Model


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 40s 156ms/step - accuracy: 0.4482 - loss: 1.3127
Epoch 2/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 42s 159ms/step - accuracy: 0.6944 - loss: 0.7376
Epoch 3/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 41s 160ms/step - accuracy: 0.7653 - loss: 0.5538
Epoch 4/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 36s 158ms/step - accuracy: 0.8687 - loss: 0.3796
Epoch 5/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 42s 164ms/step - accuracy: 0.9247 - loss: 0.2228
Epoch 6/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 39s 155ms/step - accuracy: 0.9312 - loss: 0.2091
Epoch 7/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 41s 156ms/step - accuracy: 0.9434 - loss: 0.1649
Epoch 8/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 42s 159ms/step - accuracy: 0.9466 - loss: 0.1419
Epoch 9/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 42s 166ms/step - accuracy: 0.9313 - loss: 0.2318
Epoch 10/10
230/230 ━━━━━━━━━━━━━━━━━━━━ 36s 156ms/step - accuracy: 0.8897 - loss: 0.3301
93/93 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step
Accuracy: 0.8918, Precision: 0.8922, Recall: 0.8918, F1 Score: 0.8917